# Final Project
## Jojo Duray
### 12/13/2023

In [1]:
import pandas as pd
import numpy as np
import altair as alt
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

#### Q1  
Read in the data, call the dataframe "s"  and check the dimensions of the dataframe

In [2]:
s = pd.read_csv('social_media_usage.csv')
print(s.ndim)




2


In [3]:
s.dtypes

respid          int64
sample          int64
comp            int64
int_date       object
lang            int64
               ...   
ql1            object
ql1a           object
qc1            object
weight        float64
cellweight     object
Length: 89, dtype: object

In [4]:
s = pd.DataFrame(s)
s.head()

,respid,sample,comp,int_date,lang,state,density,usr,qs1,sex,...,income,party,partyln,hh1,hh3,ql1,ql1a,qc1,weight,cellweight
0,798,1,1,1/26/2021,1,51,5,S,,2,...,6,2,,1,,1,,,0.557187,
1,1017,1,1,1/25/2021,1,39,2,S,,1,...,5,1,,2,2,1,,,0.987209,
2,1181,1,1,1/25/2021,1,32,2,U,,2,...,8,3,2,2,2,1,,,0.925147,
3,1388,1,1,1/25/2021,1,4,3,U,,1,...,8,2,,1,,1,,,0.311080,
4,1664,1,1,1/26/2021,1,12,2,S,,2,...,7,2,,2,2,1,,,0.396548,


***

#### Q2  
Define a function called clean_sm that takes one input, x, and uses `np.where` to check whether x is equal to 1. If it is, make the value of x = 1, otherwise make it 0. Return x. Create a toy dataframe with three rows and two columns and test your function to make sure it works as expected

In [8]:
def clean_sm(x):
    return np.where(x== 1, 1, 0)

In [9]:
data = {'Column1': [1, 0, 1], 'Column2': [0, 1, 1]}
df = pd.DataFrame(data)

In [10]:
df['Column1_cleaned'] = clean_sm(df['Column1'])
df['Column2_cleaned'] = clean_sm(df['Column2'])

In [11]:
print(df.head(10))

   Column1  Column2  Column1_cleaned  Column2_cleaned
0        1        0                1                0
1        0        1                0                1
2        1        1                1                1


***

#### Q3  
Create a new dataframe called "ss".   
The new dataframe should contain a target column called sm_li which should be a binary variable ( that takes the value of 1 if it is 1 and 0 otherwise (use clean_sm to create this) which indicates whether or not the individual uses LinkedIn,  

 and the following features:  
  income (ordered numeric from 1 to 9, above 9 considered missing),  
 education (ordered numeric from 1 to 8, above 8 considered missing),  
 parent (binary),  
 married (binary),  
 female (binary),  
 and age (numeric, above 98 considered missing).  
 
  Drop any missing values. Perform exploratory analysis to examine how the features are related to the target.

In [13]:
ss= pd.DataFrame({
    'sm_li': clean_sm(s['web1h']),
    'income': np.where((s["income"] >= 1) & (s["income"] <= 9), s["income"], np.nan),
    'education' :np.where(s["educ2"] <= 8, s["educ2"], np.nan),
    'parent' : np.where(s['par'] == 'yes', 1, 0),
    'married': np.where(s['marital'] <= 3, 1, 0),
    'female': np.where(s['gender'] == 1, 1, 0),
    'age' : np.where(s['age'] <= 98, s['age'], np.nan)
})
ss.dropna(subset=['income'], inplace=True)
ss.dropna(subset=['education'], inplace=True)
ss.dropna(subset=['age'], inplace=True)
ss.head()



,sm_li,income,education,parent,married,female,age
0,0,6.0,4.0,0,0,0,77.0
1,0,5.0,3.0,0,0,1,59.0
2,0,8.0,4.0,0,1,0,60.0
3,0,8.0,8.0,0,0,1,73.0
4,1,7.0,8.0,0,1,0,65.0


In [14]:
ss.isnull().sum()

sm_li        0
income       0
education    0
parent       0
married      0
female       0
age          0
dtype: int64

In [15]:
ss.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1260 entries, 0 to 1501
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   sm_li      1260 non-null   int32  
 1   income     1260 non-null   float64
 2   education  1260 non-null   float64
 3   parent     1260 non-null   int32  
 4   married    1260 non-null   int32  
 5   female     1260 non-null   int32  
 6   age        1260 non-null   float64
dtypes: float64(3), int32(4)
memory usage: 59.1 KB


***

#### Q4  
Create a target vector (y) and feature set (X)

In [16]:
y = ss["sm_li"]
X = ss[["age", "education", "income", "parent", "female"]]

***

#### Q5  
Split the data into training and test sets. Hold out 20% of the data for testing. Explain what each new object contains and how it is used in machine learning

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    stratify=y,       # same number of target in training & test set
                                                    test_size=0.2,    # hold out 20% of data for testing
                                                    random_state=987) # set for reproducibility

The above code is splitting both the (y) target variable and the (x) features into 2 different sets of data this way we have an x, and y training set to test and make models with then a x, and y set to test our model on so it has data it has never seen before so we can see how the model does on data it has not seen before. 

***

#### Q6  
Instantiate a logistic regression model and set class_weight to balanced. Fit the model with the training data.

In [18]:
# Initialize algorithm 
lr = LogisticRegression(class_weight='balanced')

In [19]:
# Fit algorithm to training data
lr.fit(X_train, y_train)

c:\Users\jorda\anaconda3\lib\site-packages\sklearn\utils\validation.py:623: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():


LogisticRegression(class_weight='balanced')

***

#### Q7  
Evaluate the model using the testing data. What is the model accuracy for the model? Use the model to make predictions and then generate a confusion matrix from the model. Interpret the confusion matrix and explain what each number means.

In [20]:
y_pred = lr.predict(X_test)

c:\Users\jorda\anaconda3\lib\site-packages\sklearn\utils\validation.py:623: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():


***

#### Q8  
Create the confusion matrix as a dataframe and add informative column names and index names that indicate what each quadrant represents

In [21]:
pd.DataFrame(confusion_matrix(y_test, y_pred),
            columns=["Predicted negative", "Predicted positive"],
            index=["Actual negative","Actual positive"]).style.background_gradient(cmap="PiYG")

,Predicted negative,Predicted positive
Actual negative,109,59
Actual positive,20,64


***

#### Q9  
Aside from accuracy, there are three other metrics used to evaluate model performance: precision, recall, and F1 score. Use the results in the confusion matrix to calculate each of these metrics by hand. Discuss each metric and give an actual example of when it might be the preferred metric of evaluation. After calculating the metrics by hand, create a classification_report using sklearn and check to ensure your metrics match those of the classification_report.

In [40]:
## recall: TP/(TP+FN)
recall = 64/(64+20)
recall


0.7619047619047619

in recall it is focused on a high positive rate even if some negatives make it in. This is really important in scnerios like medical diagnosis you don't want to miss a true positive diagnosis and someone goes without treatment.

In [41]:
## precision: TP/(TP+FP)
precision = 64/(64+59)
precision

0.5203252032520326

In Precision the focus is a low false positive rate. This would be important in things like spam emails being caught. 

In [39]:
f1= 2*((precision * recall)/(precision + recall))
f1

0.6183574879227053

f1 scores is the score that balances between both recall and precision. Things like search engines need to do best with f1 as the degree of accuracy. 

In [22]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.84      0.65      0.73       168
           1       0.52      0.76      0.62        84

    accuracy                           0.69       252
   macro avg       0.68      0.71      0.68       252
weighted avg       0.74      0.69      0.70       252



***

#### Q10  
Use the model to make predictions. For instance, what is the probability that a high income (e.g. income=8), with a high level of education (e.g. 7), non-parent who is married female and 42 years old uses LinkedIn? How does the probability change if another person is 82 years old, but otherwise the same?

In [30]:
newdata = pd.DataFrame({
    "age": [42, 82],
    "education": [7, 7],
    "income": [8, 8],
    "female": [1, 1],
    "maried": [1, 1]
})
newdata

,age,education,income,female,maried
0,42,7,8,1,1
1,82,7,8,1,1


In [31]:
newdata["prediction_linkedin_user"] = lr.predict(newdata)

c:\Users\jorda\anaconda3\lib\site-packages\sklearn\base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- maried
Feature names seen at fit time, yet now missing:
- parent

  warnings.warn(message, FutureWarning)
c:\Users\jorda\anaconda3\lib\site-packages\sklearn\utils\validation.py:623: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():


In [32]:
newdata

,age,education,income,female,maried,prediction_linkedin_user
0,42,7,8,1,1,1
1,82,7,8,1,1,1


***